In [ ]:
import sys
sys.path.append('..')
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI

def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)


# embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_deployment, openai_api_version=openai_api_version)
embeddings =  AzureOpenAIEmbeddings(
        deployment=os.getenv("EMB_DEPLOYMENT"),
        openai_api_version=os.getenv("EMB_OPENAI_API_VERSION"),
        model=os.getenv("EMB_MODEL"),
        openai_api_key=os.getenv("EMB_OPENAI_API_KEY"),
        openai_api_base=os.getenv("EMB_OPENAI_ENDPOINT"),
        openai_api_type=os.getenv("EMB_API_TYPE"),
    )

llm_gpt = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=3500,
                        temperature=0.0)
# for key, value in os.environ.items():
#   print(f"{key}: {value}")

In [ ]:
from src.utils.cube_semantic_custom import CubeSemanticLoader
def fetch_cube_metadata(*args, **kwargs):
    try:
        # # Load document from Cube meta api
        loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
        documents = loader.load()
        # to_json()
        return documents
    except Exception as e:
        # Handle exceptions gracefully and return an error response
        print("Error in fetching metadata from cube: " + str(e))
        return 0

def create_vector_store(documents, local_vector_store_path, *args, **kwargs):
    print("Loaded documents: " + str(documents))
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(local_vector_store_path)
    print("Vector store created and saved successfully!")

def load_vector_store(vector_store_path, embeddings, *args, **kwargs):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
    
    return vectorstore


In [ ]:
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
# cube_metadata = fetch_cube_metadata()
loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
cube_metadata = loader.fetch_raw_cube_metadata()

In [ ]:
import json

filepath = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/preloaded_cube_metadata/cube_metadata.json"
#save the cube metadata to a file
with open(filepath, "w") as file:
    json.dump(cube_metadata, file)

print("Cube metadata saved successfully!")

In [ ]:
#load the cube metadata from the file and format the data into list[Document] for further processing while using 
import json

filepath = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/preloaded_cube_metadata/cube_metadata.json"

# Load the JSON data from the file
with open(filepath, "r") as file:
    data = json.load(file)
documents = loader.format_raw_cube_metadata(data)

In [ ]:
vectorstore = load_vector_store(vector_store_path, embeddings)

In [ ]:
#Controller
from langchain import PromptTemplate
start_goal_prompt = PromptTemplate(
    template="""
    You are an very experienced and expert manager. You are given following query "{goal}".
    The query may contain different objectives which needs an expertise to solve. Your job is to provide a list of objectives that is easier for the experts to serve the query.
    Ensure objectives are streamlined to minimize the overall number of tasks while addressing the objective effectively. 
    Some part of the query may not be relevant to the objectives. You can ignore those parts.
    The number of tasks should be strictly less than 10.
    """,
    input_variables=["goal", "language", "tools"],
)

Tools

Metric Discovery: Assuming we are trying to 3 metrics 
Agent: Get relevant 
Get 3 top documents 

Tasks: Metric Discovery 
1. Rewrite the and Extract specifics of metric information for better retrieval
2. Extract top 3 documents related to metric information. (Basic RAG)
3. get information about metrics and compile the results and return 

Tasks: Information regarding the metric
1. Check if metric exists
2. Query vector database to descrive about the metric (RAG)



In [32]:
import json
from crewai import Agent, Task
from langchain.tools import tool
from src.utils.llamaindex_retriever import LlamaIndexRetriever
#Will create below functions once the input is setup properly
# def create_vector_store

# def ingest_cube_meta():



@tool("Extract metric queried.")
def extract_metric_description(query, *args, **kwargs):
  """Useful to isolate metric information to be further used in the its discovery"""
  agent = Agent(
        role='Metric Isolator',
        goal=
        'Accurate Extraction of Metric name or description',
        backstory=
        "You're a Metric isolator which is responsible for extracting the metrics name from the given content.",
        llm = llm_gpt,
        allow_delegation=False)
  task = Task(
        agent=agent,
        description=
        f' The primary objective is to ensure that all the metrics are correctly isolated and extracted from the query {query}. This extracted information should be organized in a form of list.',
        expected_output="some string",
  
    )
  extracted_metrics = task.execute()

  return extracted_metrics

faiss_index_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)

def get_similar_documents(query, max_number_documents=3):
  faiss_index_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
  vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
  docs = vectorstore.similarity_search_with_relevance_scores(query, max_number_documents)
  conversational_metadata = []
  for doc in docs:
      doc = doc[0]
      meta = {'text':doc.page_content, 'table_metadata': doc.metadata}
      conversational_metadata.append(meta)
  return conversational_metadata

# @tool("Metric Discovery.")
def metric_discovery_faiss(metric_description, vectorstore, *args, **kwargs):
  """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
  # documents = fetch_cube_metadata()
  # retriever = LlamaIndexRetriever('metric discovery')
  # retriever.index_document(documents)

  # docs = vectorstore.similarity_search_with_relevance_scores(metric_description, max_number_documents)
  # conversational_metadata = []
  # for doc in docs:
  #   meta = {'text':doc.text, 'table_metadata': doc.metadata}
  #   conversational_metadata.append(meta)
  conversational_metadata = get_similar_documents(metric_description)
  agent = Agent(
        role='Data Analyst Assistant',
        goal=
        'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
        backstory=
        "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
        llm = llm_gpt,
        allow_delegation=False)  
  
  task = Task(
        agent=agent,
        description=
        """ You are responding to  question {metric_description} with answer from the Metadata provided to you as {conversational_metadata}}.
            Strictly answer the question with the information present in metadata only.
            Respond with "Sorry, the query is out of scope." if the answer is not present in metadata.
            """,
        expected_output="some string",
  
    )
  output = task.execute()

  return output



prompt_metric_discovery = f"You are given a metric description as input {metric_description}. Your task is to determine whether this specific metric exists within the context provided. Thoroughly analyze the metric description and provide a definitive answer on the existence of the metric, along with any relevant details or insights that support your conclusion."

In [33]:
faiss_index_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
query = "What is the most popular feature used by our paid subscribers"
response = metric_discovery_faiss(query, vectorstore)

In [34]:
response

'Sorry, the query is out of scope.'

In [22]:
docs = vectorstore.similarity_search("What is the most popular feature used by our paid subscribers", 3)

In [23]:
docs

[Document(page_content='Users Count, None', metadata={'table_name': 'users', 'column_name': 'users.count', 'column_data_type': 'number', 'column_title': 'Users Count', 'column_description': 'None', 'column_member_type': 'measure', 'column_values': [], 'cube_data_obj_type': 'cube'}),
 Document(page_content='Users Company, None', metadata={'table_name': 'users', 'column_name': 'users.company', 'column_data_type': 'string', 'column_title': 'Users Company', 'column_description': 'None', 'column_member_type': 'dimension', 'column_values': [], 'cube_data_obj_type': 'cube'}),
 Document(page_content='Users Created at, None', metadata={'table_name': 'users', 'column_name': 'users.created_at', 'column_data_type': 'time', 'column_title': 'Users Created at', 'column_description': 'None', 'column_member_type': 'dimension', 'column_values': [], 'cube_data_obj_type': 'cube'})]

In [21]:
docs

[(Document(page_content='Users Count, None', metadata={'table_name': 'users', 'column_name': 'users.count', 'column_data_type': 'number', 'column_title': 'Users Count', 'column_description': 'None', 'column_member_type': 'measure', 'column_values': [], 'cube_data_obj_type': 'cube'}),
  0.6682709429886676),
 (Document(page_content='Users Company, None', metadata={'table_name': 'users', 'column_name': 'users.company', 'column_data_type': 'string', 'column_title': 'Users Company', 'column_description': 'None', 'column_member_type': 'dimension', 'column_values': [], 'cube_data_obj_type': 'cube'}),
  0.6650152464551708),
 (Document(page_content='Users Created at, None', metadata={'table_name': 'users', 'column_name': 'users.created_at', 'column_data_type': 'time', 'column_title': 'Users Created at', 'column_description': 'None', 'column_member_type': 'dimension', 'column_values': [], 'cube_data_obj_type': 'cube'}),
  0.6643387473897224),
 (Document(page_content='Users City, None', metadata=

In [ ]:
@tool("Metric Discovery.")
def metric_discovery_faiss(metric_description, *args, **kwargs):
  """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
  documents = fetch_cube_metadata()
  retriever = LlamaIndexRetriever('metric discovery')
  retriever.index_document(documents)

  # rephrased_query = f"Find the metric {metric_description}"

  response_doc = retriever.query_util(metric_description, documents)
  conversational_metadata = []
  for doc in response_doc:
    meta = {'text':doc.text, 'table_metadata': doc.metadata}
    conversational_metadata.append(meta)
  agent = Agent(
        role='Data Analyst Assistant',
        goal=
        'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
        backstory=
        "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
        llm = llm_gpt,
        allow_delegation=False)  
  
  task = Task(
        agent=agent,
        description=
        """ You are responding to  question {metric_description} with answer from the Metadata provided to you as {conversational_metadata}}.
            Strictly answer the question with the information present in metadata only.
            Respond with "Sorry, the query is out of scope." if the answer is not present in metadata.
            """,
        expected_output="some string",
  
    )
  output = task.execute()

  return output

In [ ]:
query = "What is the most popular feature used by our paid subscribers"
extracted_metrics = extract_metric_description(query)

In [ ]:
extracted_metrics

In [ ]:
def get_similar_documents(query, vector_store):
  docs = vector_store.similarity_search(query)
  # take the first document as the best guess
  table_name = docs[0].metadata["table_name"]
  # Columns
  columns_question = "All available columns"
  column_docs = vector_store.similarity_search(
      columns_question,
      filter=dict(table_name=table_name),
      k=20
  )
  lines = []
  for column_doc in column_docs:
      column_title = column_doc.metadata["column_title"]
      column_name = column_doc.metadata["column_name"]
      column_data_type = column_doc.metadata["column_data_type"]
      lines.append(
          f"title: {column_title}, column name: {column_name}, datatype: {column_data_type}, member type: {column_doc.metadata['column_member_type']}"
      )
  columns = "\n".join(lines)
  print("columns :", columns)

  FAISS.similarity_search_with_relevance_scores

In [ ]:
%pip install nltk

In [ ]:
# pre process of input to make it more relevant
import nltk

# Download NLTK tokenizer data (one-time setup)
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
  # Tokenization
  tokens = nltk.word_tokenize(text.lower())
  # Stop word removal
  stopwords = nltk.corpus.stopwords.words('english')
  tokens = [token for token in tokens if token not in stopwords]
  return tokens

def reformat_question(question):
  # Define synonyms and related terms
  metrics = ["metric", "KPI", "measure"]
  sales_terms = ["sales", "revenue", "turnover"]
  central_tendency = ["average", "mean", "median"]
  
  # Preprocess the question
  preprocessed_question = preprocess_text(question)
  
  # Build the reformulated query string using question tokens
  query_string = "(" + " OR ".join(metrics) + ")"
  query_string += " AND (" + " OR ".join(sales_terms) + ")"
  query_string += " AND (" + " OR ".join(central_tendency) + ")"
  
  # Include question tokens for broader coverage
  query_string += " AND (" + " OR ".join(preprocessed_question) + ")"
  
  return query_string

# Example usage
question = "Get the metric for sales on avg"
reformulated_question = reformat_question(question)
print(f"Original question: {question}")
print(f"Reformatted question for FAISS search: {reformulated_question}")


In [ ]:
from src.utils.langchain_retriever import LangchainRetriever
langchain_retriever = LangchainRetriever(embeddings)

In [ ]:
documents = fetch_cube_metadata()
langchain_retriever.index_documents(documents)

In [ ]:
type(documents[0])

In [ ]:
subdocs = langchain_retriever.retrieve_sub_document()

In [ ]:
from langchain_community.document_loaders import TextLoader

loaders = [
    TextLoader("/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files/file1.txt"),
    TextLoader("/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files/file2.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
type(docs[0])

In [ ]:
langchain_retriever.index_documents(docs)

In [ ]:
subdocs = langchain_retriever.retrieve_sub_document("vectorstore")